
# Generando el dataset para el proyecto de clasificacion de noticias

Utilizamos los archivos csv proporcionados para crear nuestro dataset inicial (disponibles en noticias)

In [1]:
import pandas as pd
from os import listdir
from os.path import join
from tqdm import tqdm
from my_funcs import clean_and_tag
import re
from pandasql import sqldf
import numpy as np

In [40]:
pd.read_csv('./noticias/chile_2021-01.csv').head(3)

,id,country,media_outlet,url,title,text,date,year,id_journalist
0,3816243,chile,elmostrador,https://www.elmostrador.cl/dia/2021/01/01/ley-de-migracion-diputada-nunez-rn-formula-observaciones-al-tc-frente-a-re...,"Ley de Migración: diputada Núñez (RN) formula observaciones al TC frente a requerimiento ingresado por el FA, PC y PS","La diputada y vicepresidenta de Renovación Nacional, Paulina Núñez, ingresará en los próximos días un escrito ante e...",2021-01-01,2021.0,NaN
1,3816387,chile,elmostrador,https://www.elmostrador.cl/destacado/2021/01/01/2021-arranca-con-3-588-nuevos-contagios-y-52-fallecidos-por-covid-19...,2021 arranca con 3.588 nuevos contagios y 52 fallecidos por Covid-19: alza de casos es la más alta desde julio,"Según el reporte de hoy, el Ministerio de Salud informa 3.588 casos nuevos de Covid-19 (la cifra más alta desde el 5...",2021-01-01,2021.0,NaN
2,3816512,chile,elmostrador,https://www.elmostrador.cl/dia/2021/01/01/829-personas-fueron-detenidas-durante-noche-de-ano-nuevo-327-de-ellas-en-h...,829 personas fueron detenidas durante noche de Año Nuevo: 327 de ellas en horario de toque de queda,"Carabineros reportó que hasta las 06:00 am. de este viernes, efectivos policiales detuvieron a 829 personas a nivel ...",2021-01-01,2021.0,NaN


 Ya que estas noticias entregadas no se encuentran etiquetadas es necesario realizar un procesamiento previo para asignarles una categoria que nos permita entrenar y probar nuestros modelos de clasificación.
 
 ### **Categorias a utilizar**: Se utilizarán las categorias propuestas en el enunciado del trabajo 
 
* 1- Mundo
* 2- Economía
* 3- Política y Conflictos
* 4- Ciencias y Tecnología,
* 5- Catástrofes y Accidentes,
* 6- Cultura y Artes, 
* 7- Deporte,
* 8- Ecología y Planeta,
* 9- Crimen, delitos y Justicia, 
* 10- Salud

### Adoptamos dos estrategias para realizar esta tarea:

1. Asignarle una categoria a partir de la información de la url.
2. Utilizar LDA para encontrar topicos que correspondan a alguna categoria.

# 1. Etiquetado a partir de la informacion contenida en la url

Debido a la gran cantidad de noticias, propusimos inicialmente utilizar esta metodologia para disminuir el trabajo inicial y aprovechando la propia información que nos proveen los medios de prensa.

Todas las noticias en su URL vienen con una categoria propia del medio, aunque esta pueda diferir entre los medios (una misma noticias para un medio puede ser *nacional* pero para otro puede ser *cultura*) podemos clasificar incialmente utilizando esta información.

In [41]:
unique_url_tags = set()
folder = './noticias'
dfs = []
for archivo in tqdm(listdir(folder)):
    df = pd.read_csv(join(folder, archivo))
    corpus = clean_and_tag(df)
    dfs.append(corpus)
    for tag in corpus.tag.unique():
        unique_url_tags.add(tag)

100%|█████████████████████████████████████████████| 8/8 [00:03<00:00,  2.14it/s]


In [42]:
print(len(unique_url_tags))
list(unique_url_tags)[:10]

83


['ciudadanos-al-poder',
 'Deportes',
 'salud',
 'animal',
 'noticias',
 'Nacional',
 'editorial',
 'reconstitucion',
 'peru',
 'programas']

### Encontramos 83 tags distintos. Revisamos esta lista y asignamos manualmente una categoria a los url_tags que pertenecieran a alguna de las 10 categorias objetivo (este trabajo manual se encuentra en el archivo ```categorias - hojav2.csv```)

In [43]:
dm = pd.read_csv('./categorias - hojav2.csv')
dm.rename(columns={'Unnamed: 0':'tag'}, inplace=True)
vals = dm.iloc[:, 1:]
labels = vals[vals.max(axis=1)>0].idxmax(axis=1)
print(len(labels))
labels.head(5)

40


1       8-Ecología_y_Planeta
2          6-Cultura_y_Artes
3    4-Ciencias_y_Tecnología
4    3-Política_y_Conflictos
8    4-Ciencias_y_Tecnología
dtype: object

### Utilizamos un diccionario para manejar de mejor manera las *tag_url* con las categorias propuestas

In [44]:
cats = pd.DataFrame(labels)
cats = cats.join(dm['tag'])
cats.rename(columns={0:'cat'}, inplace=True)
cats_dict = dict(zip(cats['tag'], cats['cat']))
list(cats_dict.items())[:5]

[('animal', '8-Ecología_y_Planeta'),
 ('artes', '6-Cultura_y_Artes'),
 ('Autos', '4-Ciencias_y_Tecnología'),
 ('braga', '3-Política_y_Conflictos'),
 ('ciencia-tecnologia', '4-Ciencias_y_Tecnología')]

Se agrega la columna ```category``` al dataframe rellenando con Nan. 
Creamos una columna *content* que contiene el título de la noticias más el cuerpo de esta.
Esto se realizó debido a que el título contiene información sumamente importante la cual explica la idea principal de esta en muy pocas lineas de texto, por lo que utilizarla nos proporciona información útil para su clasificación.

In [45]:
df = pd.concat(dfs)
df.drop_duplicates(subset='url', keep='first', inplace=True)
df['category'] = df.apply(lambda row : cats_dict.get(row.tag, np.nan), axis=1)
#df['content'] = df['title'] + df['text']
df['content'] = df[['title', 'text']].agg('. '.join, axis=1)
df.head(3)

,id,country,media_outlet,url,title,text,date,tag,category,content
0,12872,chile,horas24,https://www.24horas.cl/nacional/gobierno-da-luz-verde-a-temporada-de-piscinas-y-anuncio-reapertura-de-tupahue-4554782,Gobierno da luz verde a temporada de piscinas y anunció reapertura de Tupahue,"Con un acto en la piscina Tupahue, ubicada en el Parque Metropolitano, el Gobierno dio luz verde a la reapertura del...",2020-12-01,nacional,NaN,"Gobierno da luz verde a temporada de piscinas y anunció reapertura de Tupahue. Con un acto en la piscina Tupahue, ub..."
3,12929,chile,horas24,https://www.24horas.cl/nacional/subsecretaria-bown-la-defensoria-de-la-ninez-es-un-organo-autonomo-y-nos-gustaria-qu...,"Subsecretaria Bown: ""La Defensoría de la Niñez es un órgano autónomo y nos gustaría que se notara un poco más""","""Entendemos que la Defensoría es un órgano autónomo y nos gustaría que se notara un poco más"". Así comentó la subsec...",2020-12-01,nacional,NaN,"Subsecretaria Bown: ""La Defensoría de la Niñez es un órgano autónomo y nos gustaría que se notara un poco más"". ""Ent..."
4,12942,chile,horas24,https://www.24horas.cl/nacional/contralor-jorge-bermudez-asume-como-presidente-de-la-junta-de-auditores-de-la-onu-45...,Contralor Jorge Bermúdez asume como presidente de la Junta de Auditores de la ONU,El contralor Jorge Bermúdez asumió este 1 de diciembre la presidencia de la Junta de Auditores de Naciones Unidas re...,2020-12-01,nacional,NaN,Contralor Jorge Bermúdez asume como presidente de la Junta de Auditores de la ONU. El contralor Jorge Bermúdez asumi...


Almacenamos 30.000 de las noticias cuyo *url_tag* era muy general o no nos servia para asignarle una categoria. A estas noticias en una segunda etapa le aplicaremos LDA  para poder etiquetarlas y asi aumentar la información de nuestro dataset inicial.

In [46]:
pd.set_option('display.max_colwidth', 120)
df = df[['id','url','title', 'content', 'category']]
unravel = df[df['category'].isna()]
(unravel.sample(n=30000, random_state=2)).to_csv('30k_sin_cat.csv')
df = df[df['category'].notna()]
final_df_link = df.drop(columns=['url'])
final_df_link.to_csv('final_df_link.csv', index=False)
final_df_link.head(3)

,id,title,content,category
14,13115,"Vacunación en Chile empezaría primer trimestre del 2021 e incluiría a 15,2 millones de personas","Vacunación en Chile empezaría primer trimestre del 2021 e incluiría a 15,2 millones de personas. Este martes, el Pr...",10-Salud
18,13183,"Sernapesca: ""Viabilidad del virus de infectar es muy baja"" tras hallazgo de COVID-19 en paquete chileno en China","Sernapesca: ""Viabilidad del virus de infectar es muy baja"" tras hallazgo de COVID-19 en paquete chileno en China. La...",10-Salud
125,614680,"Presidente de México asegura que la pandemia ""no nos ha rebasado""","Presidente de México asegura que la pandemia ""no nos ha rebasado"". El presidente de México, Andrés Manuel López Obra...",1-Mundo


Luego de esto, podemos ver que pudimos etiquetar de buena manera una cantidad considerable de noticias, pero, existen categorias las cuales no pudimos proveerles una gran cantidad de noticias y existieron otras, particularmente la categoria *5- Catástrofes y Accidentes* que no fue posible, esto es debido a que los medios de prensa no tienen en sus URL una categoria para catastrofes o accidentes.

Por otro lado de esta manera es posible que nuestro dataset este sesgado, por ejemplo, en la categoria de Salud, todas las noticias son apartir de Septiembre de 2020, fechas las cuales la gran mayoria de noticias son acerca de la pandemia del Coronavirus, por lo que noticias anteriores que quizás hablan de la influenza u otras enfermedades las cuales anteriormete eran comunes pero desde esta fecha no, no sea bien clasificadas por este metodo.


In [9]:
q="""SELECT category, count(*) FROM final_df_link GROUP BY category ORDER BY count(*) DESC;"""
url_cat_result=sqldf(q)
url_cat_result

,category,count(*)
0,1-Mundo,8609
1,7-Deporte,6708
2,4-Ciencias_y_Tecnología,4225
3,2-Economía,2636
4,3-Política_y_Conflictos,1923
5,10-Salud,333
6,6-Cultura_y_Artes,185
7,8-Ecología_y_Planeta,135
8,9-Crimen_delitos_y_Justicia,130


# 2. Aplicación de LDA a noticias no clasificadas

A las 30.000 noticias que guaradmos antes le aplicamos LDA con 20, creemos que es un buen número para poder apreciar una diferencia entre los grupos formados, debido a la gran cantidad de noticias a analizar

*alternativa: tomar una muestra aleatoria de unos 10k items y buscar el número de tópicos que entregue la mejor coherencia*

In [10]:
import re
from pprint import pprint
import nltk;
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
import string
# spacy for lemmatization
import spacy
# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
df = pd.read_csv('./30k_sin_cat.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head(3)

,id,url,title,content,category
0,7329456,https://www.meganoticias.cl/tendencias/320596-adamari-lopez-adelgaza-1ab.html,"Adamari López y su lucha contra el sobrepeso: ""He sido como una montaña rusa que sube y baja""","Adamari López y su lucha contra el sobrepeso: ""He sido como una montaña rusa que sube y baja"". La actriz y presentad...",NaN
1,181069,https://www.biobiochile.cl/noticias/nacional/region-metropolitana/2020/09/18/encuentran-cuerpo-de-medico-en-rio-san-...,Encuentran cuerpo de médico en río San José de Maipo: llevaba un mes desaparecido,Encuentran cuerpo de médico en río San José de Maipo: llevaba un mes desaparecido. En las últimas horas fue encontra...,NaN
2,4829009,https://www.latercera.com/nacional/noticia/intendente-de-la-rm-por-paro-de-camioneros-interrumpir-la-libre-circulaci...,Intendente de la RM por paro de camioneros: “Interrumpir la libre circulación de vías que son públicas es un delito”,Intendente de la RM por paro de camioneros: “Interrumpir la libre circulación de vías que son públicas es un delito”...,NaN


In [12]:
def sent_to_words(sentences):
    for sentence in tqdm(sentences, desc="sentences to words"):
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [13]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
stop_words = stopwords.words('spanish')
nlp = spacy.load('es_core_news_md', disable=['parser', 'ner'])
def remove_stopwords(texts):
    desc = "removing stopwords"
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in tqdm(texts, desc=desc)]

def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in tqdm(texts, desc="making bigrams")]

def make_trigrams(texts, bigram_mod, trigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in tqdm(texts, desc="making trigrams")]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in tqdm(texts, desc="lemmatization"):
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
def get_ngrams(data_words):
    print('getting n-grams...')
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return bigram, trigram, bigram_mod, trigram_mod

In [15]:
def get_lemmatized_data(df):
    data = df.content.values.tolist()
    data = [re.sub(r'\s+', ' ', sent) for sent in tqdm(data, desc="replacing whitespaces")]
    data_words = list(sent_to_words(data))
    bigram, trigram, bigram_mod, trigram_mod = get_ngrams(data_words)
    data_words_nostops = remove_stopwords(data_words)
    data_words_bigrams = make_bigrams(data_words_nostops, bigram_mod)
    data_lemmatized = lemmatization(data_words_bigrams,
                                    allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    return data_lemmatized

In [16]:
import pickle
try:
    with open ('./data_lemmatized', 'rb') as fp:
        data_lemmatized = pickle.load(fp)
except FileNotFoundError:
    print('data_lemmatized not found...')
    data_lemmatized = get_lemmatized_data(df)
    with open('./data_lemmatized', 'wb') as fp:
        pickle.dump(data_lemmatized, fp)

### Entrenamiento del modelo de tópico con LDA

- Entrenaremos un primer modelo de tópicos buscando un modelo con 20 tópicos. 

In [17]:
model_file = "./models/model_lda_20_topics.mm"
num_topics = 20
#lda_model = gensim.models.ldamodel.LdaModel.load("./models/model_lda_20_topics.mm", )

In [18]:
try:
    lda_model = gensim.models.ldamodel.LdaModel.load("./models/model_lda_20_topics.mm")
    id2word = corpora.Dictionary.load(model_file + ".id2word")  
except FileNotFoundError:
    id2word = corpora.Dictionary(data_lemmatized) 
    lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=num_topics,
                                           chunksize=100,
                                           passes=10,
                                           per_word_topics=True,
                                                   workers=4)
    lda_model.save(model_file)
texts = data_lemmatized
#corpus = [id2word.doc2bow(text) for text in tqdm(texts)]
corpus = [id2word.doc2bow(text) for text in tqdm(texts)]

100%|███████████████████████████████████| 30000/30000 [00:03<00:00, 9341.13it/s]


In [21]:
def format_topics_documents(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(tqdm(ldamodel[corpus])):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [33]:
df_dominant_topic = pd.read_csv('./df_dominant_topic.csv')
df_dominant_topic = df_dominant_topic[['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'text_list']]
df_dominant_topic.to_csv('./df_dominant_topic.csv', index=False)

In [34]:
try:
    df_dominant_topic = pd.read_csv('./df_dominant_topic.csv')
except FileNotFoundError:
    df_topic_sents_keywords = format_topics_documents(lda_model, corpus, texts)
    # Format
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.to_csv('./df_dominant_topic.csv', index=False)
finally:    
    display(df_dominant_topic.head(20))

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,text_list
0,0,6.0,0.6294,"post_on, instagrama_post, view_this, post_shared, shared_by, ano, by, hijo, hacer, vida","['adamari_lopez', 'luchar', 'sobrepeso', 'montana_rusa', 'subir', 'bajo', 'actriz', 'presentadoro', 'vencio', 'cance..."
1,1,12.0,0.6791,"hecho, vehiculo, lugar, hombre, encontrar, persona, detenido, ano, joven, sujeto","['encontrar', 'cuerpo', 'medico', 'llevar', 'mes', 'desaparecido', 'ultimas_hora', 'encontrado', 'maipo', 'cuerpo', ..."
2,2,15.0,0.3296,"hacer, decir, ver, creer, querer, gente, mas, persona, dar, tambien","['intendente', 'paro', 'camionero', 'interrumpir', 'libre', 'circulacion', 'via', 'publica', 'delito', 'intendente',..."
3,3,8.0,0.3821,"servicio, producto, persona, encontrar, usuario, sistema, contar, realizar, empresa, hora","['dentro', 'dar', 'conocer', 'experiencia', 'usuario', 'navegar', 'consola', 'compartido', 'caracteristica', 'proxim..."
4,4,7.0,0.4914,"chileno, ano, nuevo, primero, argentino, obra, mundo, serie, temporada, historia","['homenajear', 'justicia', 'espada_acuna', 'primero', 'aniversario', 'nacimiento', 'justicia', 'mén', 'dedicado', 'd..."
5,5,6.0,0.8272,"post_on, instagrama_post, view_this, post_shared, shared_by, ano, by, hijo, hacer, vida","['rafael_araneda', 'marcela_vacarezza', 'presentar', 'hijo', 'redes_social', 'animador', 'television', 'rafael_arane..."
6,6,4.0,0.6721,"vacuna, salud, persona, dosis, primero, virus, paciente, decir, enfermedad, coronavirus","['escapista', 'variante', 'africano', 'podrio', 'disminuir', 'eficacia', 'vacuna', 'segunda_ola', 'contagio', 'sudaf..."
7,7,16.0,0.3591,"carabinero, persona, policial, general, violencia, detenido, hecho, personal, institucion, publico","['estruendo', 'alertar', 'comuna', 'rm', 'descartar', 'explosion', 'fabrico', 'domingo', 'fuerte_estruendo', 'alerto..."
8,8,4.0,0.7228,"vacuna, salud, persona, dosis, primero, virus, paciente, decir, enfermedad, coronavirus","['detalle', 'proceso', 'vacunacion', 'deposito', 'nivel', 'nacional', 'tarjeta', 'segundo', 'dosis', 'hoy', 'alegrio..."
9,9,15.0,0.7055,"hacer, decir, ver, creer, querer, gente, mas, persona, dar, tambien","['padre', 'separar', 'parte', 'llegar', 'terraza', 'cafe', 'reunir yo', 'cliente', 'empece', 'ver', 'semana_pasado',..."


In [35]:
df_topics = pd.concat([df.drop(columns=['category']), df_dominant_topic[['Dominant_Topic', 'Keywords', 'text_list']]], axis=1)

In [36]:
df_topics['Dominant_Topic'] = df_topics['Dominant_Topic'].astype(int)
df_topics.head(3)

,id,url,title,content,Dominant_Topic,Keywords,text_list
0,7329456,https://www.meganoticias.cl/tendencias/320596-adamari-lopez-adelgaza-1ab.html,"Adamari López y su lucha contra el sobrepeso: ""He sido como una montaña rusa que sube y baja""","Adamari López y su lucha contra el sobrepeso: ""He sido como una montaña rusa que sube y baja"". La actriz y presentad...",6,"post_on, instagrama_post, view_this, post_shared, shared_by, ano, by, hijo, hacer, vida","['adamari_lopez', 'luchar', 'sobrepeso', 'montana_rusa', 'subir', 'bajo', 'actriz', 'presentadoro', 'vencio', 'cance..."
1,181069,https://www.biobiochile.cl/noticias/nacional/region-metropolitana/2020/09/18/encuentran-cuerpo-de-medico-en-rio-san-...,Encuentran cuerpo de médico en río San José de Maipo: llevaba un mes desaparecido,Encuentran cuerpo de médico en río San José de Maipo: llevaba un mes desaparecido. En las últimas horas fue encontra...,12,"hecho, vehiculo, lugar, hombre, encontrar, persona, detenido, ano, joven, sujeto","['encontrar', 'cuerpo', 'medico', 'llevar', 'mes', 'desaparecido', 'ultimas_hora', 'encontrado', 'maipo', 'cuerpo', ..."
2,4829009,https://www.latercera.com/nacional/noticia/intendente-de-la-rm-por-paro-de-camioneros-interrumpir-la-libre-circulaci...,Intendente de la RM por paro de camioneros: “Interrumpir la libre circulación de vías que son públicas es un delito”,Intendente de la RM por paro de camioneros: “Interrumpir la libre circulación de vías que son públicas es un delito”...,15,"hacer, decir, ver, creer, querer, gente, mas, persona, dar, tambien","['intendente', 'paro', 'camionero', 'interrumpir', 'libre', 'circulacion', 'via', 'publica', 'delito', 'intendente',..."


# Análisis manual

In [24]:
from my_funcs import get_df_keywords
get_df_keywords(lda_model, num_topics)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,agua,region,caso,candidato,vacuna,proyecto,post_on,chileno,servicio,colegio,persona,presidente,hecho,ano,mujer,hacer,carabinero,alcalde,plebiscito,caso
1,ambiental,zona,publico,partido,salud,gobierno,instagrama_post,ano,producto,ano,paso,ministro,vehiculo,millón,menor,decir,persona,sector,voto,nuevo
2,pueblo,hora,delito,primario,persona,ministro,view_this,nuevo,persona,educacion,comuna,diputado,lugar,nuevo,genero,ver,policial,incendio,votar,contagio
3,comunidad,biobio,fiscal,independiente,dosis,ley,post_shared,primero,encontrar,profesor,medida,politico,hombre,mas,violencia,creer,general,trabajador,proceso,total
4,pueblos_originario,rio,fiscalia,candidatura,primero,diputado,shared_by,argentino,usuario,estudiante,fase,ex,encontrar,mayor,adolescente,querer,violencia,vecino,presidente,cifra
5,territorio,nacional,ministerio,alcalde,virus,presidente,ano,obra,sistema,clase,salud,cargo,persona,estudio,nino,gente,detenido,municipio,politico,salud
6,tierra,eclipse,hecho,acuerdo,paciente,hacer,by,mundo,contar,tambien,cuarentena,parlamentario,detenido,tambien,servicio,mas,hecho,común,electoral,persona
7,derecho,sismo,tribunal,ir,decir,parlamentario,hijo,serie,realizar,salud,plan,senador,ano,desarrollo,edad,persona,personal,trabajar,octubre,nivel
8,mapuche,total,ano,constituyente,enfermedad,comision,hacer,temporada,empresa,alumno,semana,jefe,joven,empresa,talcahuano,dar,institucion,trabajo,apruebo,aumento
9,isla,región,imputado,politico,coronavirus,retiro,vida,historia,hora,nino,permiso,decir,sujeto,nivel,familia,tambien,publico,bombero,votacion,región



    1- Mundo
    2- Economía
    3- Política y Conflictos
    4- Ciencias y Tecnología,
    5- Catástrofes y Accidentes,
    6- Cultura y Artes,
    7- Deporte,
    8- Ecología y Planeta,
    9- Crimen, delitos y Justicia,
    10- Salud


Analizando los 20 tópicos determinamos que cada uno de estos corresponde a las siguentes categorías propuestas.

Algunos tópicos como el 6, 13 y 19 serán procesados con LDA nuevamente para poder mejorar su clasificación debido a que en esta primera instancia no fue posible clasificarlos de buena manera.

In [25]:
url_cat_result

,category,count(*)
0,1-Mundo,8609
1,7-Deporte,6708
2,4-Ciencias_y_Tecnología,4225
3,2-Economía,2636
4,3-Política_y_Conflictos,1923
5,10-Salud,333
6,6-Cultura_y_Artes,185
7,8-Ecología_y_Planeta,135
8,9-Crimen_delitos_y_Justicia,130


In [37]:
# nos importan mas aquellas categorias menos numerosas
cats_dict = {1 : '8-Ecología_y_Planeta', # importante
             2 : '9-Crimen_delitos_y_Justicia', # importante
             3 : '3-Política_y_Conflictos',
             4 : '10-Salud', # importante 
             5 : '3-Política_y_Conflictos',
             7 : '6-Cultura_y_Artes', # importante
             11 : '3-Política_y_Conflictos',
             12 : '9-Crimen_delitos_y_Justicia', #importante
             14 : '9-Crimen_delitos_y_Justicia', #importante
             16 : '9-Crimen_delitos_y_Justicia', #importante
             17 : '5-Catástrofes_y_Accidentes', # importante
             18 : '3-Política_y_Conflictos',}

# el resto aun sin categoria
for i in range(0, num_topics):
    if i not in cats_dict.keys():
        cats_dict[i] = np.nan
        
cats_dict
df_topics['category'] = df_topics.apply(lambda row : cats_dict[row.Dominant_Topic], axis=1)
df_topics.head(10)

,id,url,title,content,Dominant_Topic,Keywords,text_list,category
0,7329456,https://www.meganoticias.cl/tendencias/320596-adamari-lopez-adelgaza-1ab.html,"Adamari López y su lucha contra el sobrepeso: ""He sido como una montaña rusa que sube y baja""","Adamari López y su lucha contra el sobrepeso: ""He sido como una montaña rusa que sube y baja"". La actriz y presentad...",6,"post_on, instagrama_post, view_this, post_shared, shared_by, ano, by, hijo, hacer, vida","['adamari_lopez', 'luchar', 'sobrepeso', 'montana_rusa', 'subir', 'bajo', 'actriz', 'presentadoro', 'vencio', 'cance...",NaN
1,181069,https://www.biobiochile.cl/noticias/nacional/region-metropolitana/2020/09/18/encuentran-cuerpo-de-medico-en-rio-san-...,Encuentran cuerpo de médico en río San José de Maipo: llevaba un mes desaparecido,Encuentran cuerpo de médico en río San José de Maipo: llevaba un mes desaparecido. En las últimas horas fue encontra...,12,"hecho, vehiculo, lugar, hombre, encontrar, persona, detenido, ano, joven, sujeto","['encontrar', 'cuerpo', 'medico', 'llevar', 'mes', 'desaparecido', 'ultimas_hora', 'encontrado', 'maipo', 'cuerpo', ...",9-Crimen_delitos_y_Justicia
2,4829009,https://www.latercera.com/nacional/noticia/intendente-de-la-rm-por-paro-de-camioneros-interrumpir-la-libre-circulaci...,Intendente de la RM por paro de camioneros: “Interrumpir la libre circulación de vías que son públicas es un delito”,Intendente de la RM por paro de camioneros: “Interrumpir la libre circulación de vías que son públicas es un delito”...,15,"hacer, decir, ver, creer, querer, gente, mas, persona, dar, tambien","['intendente', 'paro', 'camionero', 'interrumpir', 'libre', 'circulacion', 'via', 'publica', 'delito', 'intendente',...",NaN
3,6343140,https://www.latercera.com/que-pasa/noticia/asi-es-playstation-5-por-dentro-dan-a-conocer-la-experiencia-que-tendra-e...,Así es PlayStation 5 por dentro: dan a conocer la experiencia que tendrá el usuario al navegar en la consola,Así es PlayStation 5 por dentro: dan a conocer la experiencia que tendrá el usuario al navegar en la consola. Sony I...,8,"servicio, producto, persona, encontrar, usuario, sistema, contar, realizar, empresa, hora","['dentro', 'dar', 'conocer', 'experiencia', 'usuario', 'navegar', 'consola', 'compartido', 'caracteristica', 'proxim...",NaN
4,3808041,https://www.meganoticias.cl/nacional/324205-doodle-google-homenaje-justicia-espada-acuna-mena-ingeniera-civl-mgx17.html,"Google homenajea a Justicia Espada Acuña, la primera ingeniera civil del país","Google homenajea a Justicia Espada Acuña, la primera ingeniera civil del país. Para conmemorar el 128° aniversario d...",7,"chileno, ano, nuevo, primero, argentino, obra, mundo, serie, temporada, historia","['homenajear', 'justicia', 'espada_acuna', 'primero', 'aniversario', 'nacimiento', 'justicia', 'mén', 'dedicado', 'd...",6-Cultura_y_Artes
5,7355093,https://www.meganoticias.cl/tendencias/313371-rafael-araneda-y-marcela-vacarezza-presentaron-a-su-hijo-benjamin-rafa...,Rafael Araneda y Marcela Vacarezza presentaron a su hijo Benjamín Rafael,"Rafael Araneda y Marcela Vacarezza presentaron a su hijo Benjamín Rafael. En redes sociales, los animadores de telev...",6,"post_on, instagrama_post, view_this, post_shared, shared_by, ano, by, hijo, hacer, vida","['rafael_araneda', 'marcela_vacarezza', 'presentar', 'hijo', 'redes_social', 'animador', 'television', 'rafael_arane...",NaN
6,6321224,https://www.latercera.com/que-pasa/noticia/mutacion-escapista-variante-africana-del-coronavirus-podria-disminuir-la-...,“Mutación escapista”: Variante africana del coronavirus podría disminuir la eficacia de la vacuna,“Mutación escapista”: Variante africana del coronavirus podría disminuir la eficacia de la vacuna. La segunda ola de...,4,"vacuna, salud, persona, dosis, primero, virus, paciente, decir, enfermedad, coronavirus","['escapista', 'variante', 'africano', 'podrio', 'disminuir', 'eficacia', 'vacuna', 'segunda_ola', 'contagio', 'sudaf...",10-Salud
7,13387486,https://www.e

In [38]:
final_df_topics = df_topics[['id','url','title', 'content', 'category']]
final_df_topics = final_df_topics[final_df_topics['category'].notna()]
final_df_topics.drop(columns=['url'], inplace=True)
final_df_topics.to_csv('final_df_topic.csv', index=False)
final_df_topics.head(3)

,id,title,content,category
1,181069,Encuentran cuerpo de médico en río San José de Maipo: llevaba un mes desaparecido,Encuentran cuerpo de médico en río San José de Maipo: llevaba un mes desaparecido. En las últimas horas fue encontra...,9-Crimen_delitos_y_Justicia
4,3808041,"Google homenajea a Justicia Espada Acuña, la primera ingeniera civil del país","Google homenajea a Justicia Espada Acuña, la primera ingeniera civil del país. Para conmemorar el 128° aniversario d...",6-Cultura_y_Artes
6,6321224,“Mutación escapista”: Variante africana del coronavirus podría disminuir la eficacia de la vacuna,“Mutación escapista”: Variante africana del coronavirus podría disminuir la eficacia de la vacuna. La segunda ola de...,10-Salud


In [47]:
final_df_link = pd.read_csv('final_df_link.csv')
final_df_link.head(3)

,id,title,content,category
0,13115,"Vacunación en Chile empezaría primer trimestre del 2021 e incluiría a 15,2 millones de personas","Vacunación en Chile empezaría primer trimestre del 2021 e incluiría a 15,2 millones de personas. Este martes, el Pr...",10-Salud
1,13183,"Sernapesca: ""Viabilidad del virus de infectar es muy baja"" tras hallazgo de COVID-19 en paquete chileno en China","Sernapesca: ""Viabilidad del virus de infectar es muy baja"" tras hallazgo de COVID-19 en paquete chileno en China. La...",10-Salud
2,614680,"Presidente de México asegura que la pandemia ""no nos ha rebasado""","Presidente de México asegura que la pandemia ""no nos ha rebasado"". El presidente de México, Andrés Manuel López Obra...",1-Mundo


In [48]:
q="""SELECT category, count(*) FROM final_df_link GROUP BY category ORDER BY count(*) DESC;"""
result=sqldf(q)
result

,category,count(*)
0,1-Mundo,8609
1,7-Deporte,6708
2,4-Ciencias_y_Tecnología,4225
3,2-Economía,2636
4,3-Política_y_Conflictos,1923
5,10-Salud,333
6,6-Cultura_y_Artes,185
7,8-Ecología_y_Planeta,135
8,9-Crimen_delitos_y_Justicia,130


In [49]:
q="""SELECT category, count(*) FROM final_df_topics GROUP BY category ORDER BY count(*) DESC;"""
result=sqldf(q)
result

,category,count(*)
0,9-Crimen_delitos_y_Justicia,8083
1,3-Política_y_Conflictos,5732
2,10-Salud,1571
3,5-Catástrofes_y_Accidentes,1245
4,8-Ecología_y_Planeta,545
5,6-Cultura_y_Artes,357


In [50]:
final_df = pd.concat([final_df_link, final_df_topics]).reset_index().drop(columns=['index'])

In [51]:
q="""SELECT category, count(*) FROM final_df GROUP BY category ORDER BY count(*) DESC;"""
result=sqldf(q)
result

,category,count(*)
0,1-Mundo,8609
1,9-Crimen_delitos_y_Justicia,8213
2,3-Política_y_Conflictos,7655
3,7-Deporte,6708
4,4-Ciencias_y_Tecnología,4225
5,2-Economía,2636
6,10-Salud,1904
7,5-Catástrofes_y_Accidentes,1245
8,8-Ecología_y_Planeta,680
9,6-Cultura_y_Artes,542


In [53]:
final_df.to_csv('final_df_paso1.csv', index=False)

In [54]:
df_sin_topic = df_topics[df_topics.category.isna()]
len(df_sin_topic)

12467

Cerca del 40% del dataset sin categoría (post revision de urls) quedó con un tópico al que no pudimos asignarle una de nuestras categorías objetivo.

Adoptaremos dos enfoques diferentes para tratar de etiquetar correctamente estas noticias, priorizando las categorías menos numerosas con el objetivo de obtener un dataset relativamente balanceado.

### LDA al subset completo

In [55]:
df_sin_topic.to_csv('df_sin_topic.csv', index=False)

# Clasificacion de subtopicos en el cuadernillo 2. ***paso_2_re_extraccion_lda.ipynb***